In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils import getXY

### I. Data Preprocessing

In [2]:
#load data
data = pd.read_csv('data/cleanData.csv')

In [3]:
#double check nan
data.isna().sum()

course_id      0
cc_by_ip     647
certified      0
LoE            6
YoB            0
gender        23
nevents        0
grade          0
dtype: int64

In [4]:
#drop na columns
data_clean = data.dropna()
data_clean.shape

(2789511, 8)

In [5]:
data_clean.course_id.unique()

array(['PH', 'ER', 'HKS', '1368', 'MUS', 'Hum', 'MCB', 'SW', 'HUM',
       'Urban', 'CS', 'HLS', 'CHEM', 'SOC', 'SPU', 'VJx', 'AmPoX', 'HDS',
       'GSE', 'BUS', 'HHP', 'CB', 'GSD', 'ENGSCI', 'AT', 'USW', 'HSPH',
       'CalcAPL', 'AI', 'FC', 'CHN', 'EMC'], dtype=object)

In [6]:
data_clean.head()

,course_id,cc_by_ip,certified,LoE,YoB,gender,nevents,grade
0,PH,DE,True,b,1980,m,537,1.00
1,PH,PT,True,m,1969,m,2427,0.98
2,PH,ES,True,hs,1993,m,4667,0.96
3,PH,US,True,b,1988,f,2197,0.70
4,PH,US,True,jhs,1999,m,2320,0.71


In [7]:
# data, response_var, clip_cols, clip_vals, dummy_cols, quant_cols, random_state = 208
#params
response = ['course_id']
clip_cols = ['YoB', 'grade']
clip_vals = [[2013-80, 2018-10], [0, 1]]
dummy_cols = ['cc_by_ip', 'LoE', 'gender']
quant_cols = ['YoB', 'nevents']
X, Y = getXY(data_clean, response, clip_cols, clip_vals, dummy_cols, quant_cols)

/Users/KaReeNaaah/anaconda3/envs/109b/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Normalizaton complete.
Dummify complete.


In [10]:
#train test split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.20, random_state = 208)

## II. Baseline Model - Cross-Validated Logistic Regression

In [16]:
#import packages 
from sklearn.linear_model import LogisticRegressionCV

In [24]:
#fit baseline model
# mBaseline = LogisticRegressionCV(cv = 5, multi_class = 'multinomial').fit(trainX, trainY.course_id.values)

/Users/KaReeNaaah/anaconda3/envs/109b/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


KeyboardInterrupt: 

### III. Model

In [ ]:
#import packages
import tensorflow as tf
from tensorflow.contrib import keras
from keras.layers import Dense
from keras.models import Sequential,Model
from keras.optimizers import SGD, Adagrad, RMSprop
from keras import backend as K
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
trainX.shape

In [ ]:
trainY.shape

In [ ]:
def getModel(optimizer):
    model = Sequential()
    model.add(Dense(20, activation = 'relu', kernel_initializer='normal'))
    model.add(Dense(10, activation = 'relu', kernel_initializer='normal'))
    model.add(Dense(5, activation = 'relu', kernel_initializer='normal'))
    model.add(Dense(1, activation = 'relu', kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer = optimizer)
    return model

In [ ]:
#params
eta = 0.01
momentum = 0.8
decay = 0.0001
nEpoch = 10

#non-private optimizers
npSGD = SGD(lr = eta, momentum = momentum, decay = decay)
npAdaGrade = Adagrad(lr = eta, decay = decay)
npRMSprop = RMSprop(lr = eta, decay = decay)

In [ ]:
npMSGD = getModel(npSGD)
npMSGD.fit(trainX.values, trainY.values, epochs = nEpoch, 
           validation_split = 0.20, verbose = True)